# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    print(root, filepath)
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
/home/workspace/event_data /home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
# (artist_name,title ,user ,user_id ,session_id ,item_in_session)
line[0], line[9], line[2]+' '+line[4], line[10], line[8], line[3]

('Alicia Keys',
 'PUT',
 'Ryan 1',
 'NextSong',
 'San Jose-Sunnyvale-Santa Clara, CA',
 'M')

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydb
        WITH replication = {'class': 'SimpleStrategy', 'replication_factor':1}
        """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## Query 1:  Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
filtered_music_library = """
            CREATE TABLE IF NOT EXISTS music_library
            (
                session_id int,
                item_in_session int,
                artist_name text,
                title text,
                length decimal,
                PRIMARY KEY(session_id, item_in_session)
            )
            """
try:
    session.execute(filtered_music_library)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        populate_music_library = "INSERT INTO music_library (session_id, item_in_session, artist_name, title, length)"
        populate_music_library = populate_music_library + "VALUES (%s,%s,%s,%s,%s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(populate_music_library, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
verify_music_library = "SELECT artist_name, title, length FROM music_library WHERE session_id = 338 AND item_in_session = 4"

try:
    result = session.execute(verify_music_library)
except Exception as e:
    print(e)

for row in result:
    print(row.artist_name, row.title, row.length, sep=' || ')

Faithless || Music Matters (Mark Knight Dub) || 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Give the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

create_sorted_music_library = """
            CREATE TABLE IF NOT EXISTS sorted_music_library
            (
                user_id int,
                session_id int,
                item_in_session int,
                artist_name text,
                title text,
                user text,
                PRIMARY KEY(user_id, session_id, item_in_session)
            )
            """
try:
    session.execute(create_sorted_music_library)
except Exception as e:
    print(e)


In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        populate_sorted_music_library = "INSERT INTO sorted_music_library (user_id ,session_id ,item_in_session, artist_name,title ,user)"
        populate_sorted_music_library = populate_sorted_music_library + "VALUES (%s,%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(populate_sorted_music_library, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[2]+' '+line[4]))

In [15]:
test_music_library = "SELECT artist_name, title, user FROM sorted_music_library WHERE user_id=10 AND session_id=182" 

try:
    result = session.execute(test_music_library)
except Exception as e:
    print(e)

for row in result:
    print(row.artist_name, row.title, row.user, sep=' || ')

Down To The Bone || Keep On Keepin' On || F Cruz
Three Drives || Greece 2000 || F Cruz
Sebastien Tellier || Kilometer || F Cruz
Lonnie Gordon || Catch You Baby (Steve Pitron & Max Sanna Radio Edit) || F Cruz


In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

filtered_all_user_table = """
            CREATE TABLE IF NOT EXISTS all_hands_against_his_own_table
            (
                title text,
                user_id int,
                user text,
                PRIMARY KEY(title, user_id)
            )
            """
try:
    session.execute(filtered_all_user_table)
except Exception as e:
    print(e)
                    

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        populate_all_user_table = "INSERT INTO all_hands_against_his_own_table (title, user_id, user)"
        populate_all_user_table = populate_all_user_table + "VALUES (%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(populate_all_user_table, (line[9], int(line[10]), line[2]+' '+line[4]))
        

In [18]:
# Add in the SELECT statement to verify the data was entered into the table

test_query = "SELECT user FROM all_hands_against_his_own_table WHERE title='Keep On Keepin'' On'"

try:
    result = session.execute(test_query)
except Exception as e:
    print(e)

for row in result:
    print(row.user)

F Cruz


### Drop the tables before closing out the sessions

In [19]:
## Drop the table

tables = ['music_library', 'sorted_music_library', 'all_hands_against_his_own_table']
for table in tables:
    try:
        drop_table = 'DROP TABLE '+table
        session.execute(drop_table)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()